In [1]:
import numpy as np
import warnings 
import sys 
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

C:\Users\User\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\User\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\User\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/enb.arff')
data = pd.DataFrame(data)
data.head()

,Relative_compactness,X1,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,0.0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,0.0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,0.0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5.0,0.0,0.0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2.0,0.0,0.0,20.84,28.28


In [5]:
X = data.iloc[:,:-2].values
y = data.iloc[:,-2:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=100)

(768, 8) (768, 2)


In [6]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(537, 8) (537, 2) (231, 8) (231, 2)


In [7]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 768
    p = 8

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

### Bidirectional Model

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [12]:
ENS_COUNT = 2

In [13]:
def create_c1node(X_feed,y_now):
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='he_normal'))
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='he_normal'))
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    
    return model

### Forward Ensemble

In [14]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_forward = []
    
    X_feed = X_train.copy()
    y_t = y_train.copy()
    
    for i in range(y_t.shape[1]):
        print('Training chain node ',i)
        y_now = y_t[:,[i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_forward.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_fwd.append(chain_forward)

Training Ensemble 0 - 

Training chain node  0
Shapes - 
 X = (537, 8) 
 Y = (537, 1)
Epoch 1/500
537/537 [==============================] - 1s - loss: 0.6407 - mean_squared_error: 0.6407     
Epoch 2/500
537/537 [==============================] - 0s - loss: 0.1733 - mean_squared_error: 0.1733     
Epoch 3/500
537/537 [==============================] - 0s - loss: 0.1159 - mean_squared_error: 0.1159     
Epoch 4/500
537/537 [==============================] - 0s - loss: 0.0881 - mean_squared_error: 0.0881     
Epoch 5/500
537/537 [==============================] - 0s - loss: 0.0715 - mean_squared_error: 0.0715     
Epoch 6/500
537/537 [==============================] - 0s - loss: 0.0659 - mean_squared_error: 0.0659     
Epoch 7/500
537/537 [==============================] - 0s - loss: 0.0605 - mean_squared_error: 0.0605     
Epoch 8/500
537/537 [==============================] - 0s - loss: 0.0578 - mean_squared_error: 0.0578     
Epoch 9/500
537/537 [==============================] - 0s 

537/537 [==============================] - 0s - loss: 0.0058 - mean_squared_error: 0.0058     
Epoch 76/500
537/537 [==============================] - 0s - loss: 0.0058 - mean_squared_error: 0.0058     
Epoch 77/500
537/537 [==============================] - 0s - loss: 0.0055 - mean_squared_error: 0.0055     
Epoch 78/500
537/537 [==============================] - 0s - loss: 0.0052 - mean_squared_error: 0.0052     
Epoch 79/500
537/537 [==============================] - 0s - loss: 0.0045 - mean_squared_error: 0.0045     
Epoch 80/500
537/537 [==============================] - 0s - loss: 0.0042 - mean_squared_error: 0.0042     
Epoch 81/500
537/537 [==============================] - 0s - loss: 0.0041 - mean_squared_error: 0.0041     
Epoch 82/500
537/537 [==============================] - 0s - loss: 0.0041 - mean_squared_error: 0.0041     
Epoch 83/500
537/537 [==============================] - 0s - loss: 0.0041 - mean_squared_error: 0.0041     
Epoch 84/500
537/537 [===================

537/537 [==============================] - 0s - loss: 7.8840e-04 - mean_squared_error: 7.8840e-04     
Epoch 149/500
537/537 [==============================] - 0s - loss: 7.0806e-04 - mean_squared_error: 7.0806e-04     
Epoch 150/500
537/537 [==============================] - 0s - loss: 7.6398e-04 - mean_squared_error: 7.6398e-04     
Epoch 151/500
537/537 [==============================] - 0s - loss: 6.8093e-04 - mean_squared_error: 6.8093e-04     
Epoch 152/500
537/537 [==============================] - 0s - loss: 6.1659e-04 - mean_squared_error: 6.1659e-04     
Epoch 153/500
537/537 [==============================] - 0s - loss: 7.3030e-04 - mean_squared_error: 7.3030e-04     
Epoch 154/500
537/537 [==============================] - 0s - loss: 6.9775e-04 - mean_squared_error: 6.9775e-04     
Epoch 155/500
537/537 [==============================] - 0s - loss: 6.0907e-04 - mean_squared_error: 6.0907e-04     
Epoch 156/500
537/537 [==============================] - 0s - loss: 6.4839e-04

537/537 [==============================] - 0s - loss: 5.4386e-04 - mean_squared_error: 5.4386e-04     
Epoch 218/500
537/537 [==============================] - 0s - loss: 4.8204e-04 - mean_squared_error: 4.8204e-04     
Epoch 219/500
537/537 [==============================] - 0s - loss: 3.3241e-04 - mean_squared_error: 3.3241e-04     
Epoch 220/500
537/537 [==============================] - 0s - loss: 4.6096e-04 - mean_squared_error: 4.6096e-04     
Epoch 221/500
537/537 [==============================] - 0s - loss: 4.0980e-04 - mean_squared_error: 4.0980e-04     
Epoch 222/500
537/537 [==============================] - 0s - loss: 3.6594e-04 - mean_squared_error: 3.6594e-04     
Epoch 223/500
537/537 [==============================] - 0s - loss: 3.2697e-04 - mean_squared_error: 3.2697e-04     
Epoch 224/500
537/537 [==============================] - 0s - loss: 4.9622e-04 - mean_squared_error: 4.9622e-04     
Epoch 225/500
537/537 [==============================] - 0s - loss: 3.6703e-04

537/537 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014     
Epoch 288/500
537/537 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 289/500
537/537 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012     
Epoch 290/500
537/537 [==============================] - 0s - loss: 7.9998e-04 - mean_squared_error: 7.9998e-04     
Epoch 291/500
537/537 [==============================] - 0s - loss: 7.5787e-04 - mean_squared_error: 7.5787e-04     
Epoch 292/500
537/537 [==============================] - 0s - loss: 8.6704e-04 - mean_squared_error: 8.6704e-04     
Epoch 293/500
537/537 [==============================] - 0s - loss: 6.2457e-04 - mean_squared_error: 6.2457e-04     
Epoch 294/500
537/537 [==============================] - 0s - loss: 4.9614e-04 - mean_squared_error: 4.9614e-04     
Epoch 295/500
537/537 [==============================] - 0s - loss: 6.6164e-04 - mean_squared_error: 6

...

537/537 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 147/500
537/537 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019             
Epoch 148/500
537/537 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Epoch 149/500
537/537 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Epoch 150/500
537/537 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 151/500
537/537 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 152/500
537/537 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 153/500
537/537 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022     
Epoch 154/500
537/537 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022     
Epoch 155/500
537/537 [==

537/537 [==============================] - 0s - loss: 9.1492e-04 - mean_squared_error: 9.1492e-04     
Epoch 221/500
537/537 [==============================] - 0s - loss: 8.1693e-04 - mean_squared_error: 8.1693e-04     
Epoch 222/500
537/537 [==============================] - 0s - loss: 9.6083e-04 - mean_squared_error: 9.6083e-04     
Epoch 223/500
537/537 [==============================] - 0s - loss: 8.5281e-04 - mean_squared_error: 8.5281e-04     
Epoch 224/500
537/537 [==============================] - 0s - loss: 8.2782e-04 - mean_squared_error: 8.2782e-04     
Epoch 225/500
537/537 [==============================] - 0s - loss: 7.1489e-04 - mean_squared_error: 7.1489e-04     
Epoch 226/500
537/537 [==============================] - 0s - loss: 8.0089e-04 - mean_squared_error: 8.0089e-04     
Epoch 227/500
537/537 [==============================] - 0s - loss: 7.6287e-04 - mean_squared_error: 7.6287e-04     
Epoch 228/500
537/537 [==============================] - 0s - loss: 7.9035e-04

537/537 [==============================] - 0s - loss: 8.2948e-04 - mean_squared_error: 8.2948e-04     
Epoch 291/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 292/500
537/537 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 293/500
537/537 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 294/500
537/537 [==============================] - 0s - loss: 9.8769e-04 - mean_squared_error: 9.8769e-04     
Epoch 295/500
537/537 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012             
Epoch 296/500
537/537 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014     
Epoch 297/500
537/537 [==============================] - 0s - loss: 8.7672e-04 - mean_squared_error: 8.7672e-04     
Epoch 298/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011       

537/537 [==============================] - 0s - loss: 5.6771e-04 - mean_squared_error: 5.6771e-04     
Epoch 362/500
537/537 [==============================] - 0s - loss: 5.3894e-04 - mean_squared_error: 5.3894e-04     
Epoch 363/500
537/537 [==============================] - 0s - loss: 4.3931e-04 - mean_squared_error: 4.3931e-04     
Epoch 364/500
537/537 [==============================] - 0s - loss: 7.1074e-04 - mean_squared_error: 7.1074e-04     
Epoch 365/500
537/537 [==============================] - 0s - loss: 6.3609e-04 - mean_squared_error: 6.3609e-04     
Epoch 366/500
537/537 [==============================] - 0s - loss: 6.9910e-04 - mean_squared_error: 6.9910e-04     
Epoch 367/500
537/537 [==============================] - 0s - loss: 6.4366e-04 - mean_squared_error: 6.4366e-04     
Epoch 368/500
537/537 [==============================] - 0s - loss: 5.2410e-04 - mean_squared_error: 5.2410e-04     
Epoch 369/500
537/537 [==============================] - 0s - loss: 5.5416e-04

537/537 [==============================] - 0s - loss: 0.0025 - mean_squared_error: 0.0025     
Epoch 433/500
537/537 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024     
Epoch 434/500
537/537 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 435/500
537/537 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 436/500
537/537 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 437/500
537/537 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 438/500
537/537 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 439/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 440/500
537/537 [==============================] - 0s - loss: 0.0032 - mean_squared_error: 0.0032     
Epoch 441/500
537/537 [==========

### Backward Ensemble

In [16]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_reverse = []
    
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print('Training data - ',i)
        y_now = y_t[:,[-i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_reverse.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_rev.append(chain_reverse)

Training Ensemble 0 - 

Training data -  1
Shapes - 
 X = (537, 8) 
 Y = (537, 1)
Epoch 1/500
537/537 [==============================] - 0s - loss: 0.5495 - mean_squared_error: 0.5495     
Epoch 2/500
537/537 [==============================] - 0s - loss: 0.2389 - mean_squared_error: 0.2389     
Epoch 3/500
537/537 [==============================] - 0s - loss: 0.1534 - mean_squared_error: 0.1534     
Epoch 4/500
537/537 [==============================] - 0s - loss: 0.1138 - mean_squared_error: 0.1138     
Epoch 5/500
537/537 [==============================] - 0s - loss: 0.0986 - mean_squared_error: 0.0986     
Epoch 6/500
537/537 [==============================] - 0s - loss: 0.0903 - mean_squared_error: 0.0903     
Epoch 7/500
537/537 [==============================] - 0s - loss: 0.0854 - mean_squared_error: 0.0854     
Epoch 8/500
537/537 [==============================] - 0s - loss: 0.0821 - mean_squared_error: 0.0821     
Epoch 9/500
537/537 [==============================] - 0s - lo

537/537 [==============================] - 0s - loss: 0.0157 - mean_squared_error: 0.0157     
Epoch 77/500
537/537 [==============================] - 0s - loss: 0.0148 - mean_squared_error: 0.0148     
Epoch 78/500
537/537 [==============================] - 0s - loss: 0.0133 - mean_squared_error: 0.0133     
Epoch 79/500
537/537 [==============================] - 0s - loss: 0.0126 - mean_squared_error: 0.0126     
Epoch 80/500
537/537 [==============================] - 0s - loss: 0.0134 - mean_squared_error: 0.0134     
Epoch 81/500
537/537 [==============================] - 0s - loss: 0.0129 - mean_squared_error: 0.0129     
Epoch 82/500
537/537 [==============================] - 0s - loss: 0.0126 - mean_squared_error: 0.0126     
Epoch 83/500
537/537 [==============================] - 0s - loss: 0.0124 - mean_squared_error: 0.0124     
Epoch 84/500
537/537 [==============================] - 0s - loss: 0.0130 - mean_squared_error: 0.0130     
Epoch 85/500
537/537 [===================

537/537 [==============================] - 0s - loss: 0.0028 - mean_squared_error: 0.0028     
Epoch 152/500
537/537 [==============================] - 0s - loss: 0.0032 - mean_squared_error: 0.0032     
Epoch 153/500
537/537 [==============================] - 0s - loss: 0.0031 - mean_squared_error: 0.0031     
Epoch 154/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 155/500
537/537 [==============================] - 0s - loss: 0.0026 - mean_squared_error: 0.0026     
Epoch 156/500
537/537 [==============================] - 0s - loss: 0.0035 - mean_squared_error: 0.0035     
Epoch 157/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 158/500
537/537 [==============================] - 0s - loss: 0.0026 - mean_squared_error: 0.0026     
Epoch 159/500
537/537 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 160/500
537/537 [==========

537/537 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 226/500
537/537 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 227/500
537/537 [==============================] - 0s - loss: 0.0033 - mean_squared_error: 0.0033     
Epoch 228/500
537/537 [==============================] - 0s - loss: 0.0028 - mean_squared_error: 0.0028     
Epoch 229/500
537/537 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 230/500
537/537 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 231/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011             
Epoch 232/500
537/537 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 233/500
537/537 [==============================] - 0s - loss: 9.9427e-04 - mean_squared_error: 9.9427e-04
Epoch 234/500
537/537 

537/537 [==============================] - 0s - loss: 7.5903e-04 - mean_squared_error: 7.5903e-04     
Epoch 298/500
537/537 [==============================] - 0s - loss: 8.2712e-04 - mean_squared_error: 8.2712e-04
Epoch 299/500
537/537 [==============================] - 0s - loss: 9.2085e-04 - mean_squared_error: 9.2085e-04     
Epoch 300/500
537/537 [==============================] - 0s - loss: 8.4350e-04 - mean_squared_error: 8.4350e-04     
Epoch 301/500
537/537 [==============================] - 0s - loss: 8.7237e-04 - mean_squared_error: 8.7237e-04
Epoch 302/500
537/537 [==============================] - 0s - loss: 6.2242e-04 - mean_squared_error: 6.2242e-04     
Epoch 303/500
537/537 [==============================] - 0s - loss: 5.2890e-04 - mean_squared_error: 5.2890e-04     
Epoch 304/500
537/537 [==============================] - 0s - loss: 6.3467e-04 - mean_squared_error: 6.3467e-04     
Epoch 305/500
537/537 [==============================] - 0s - loss: 6.7995e-04 - mean_sq

...

537/537 [==============================] - 0s - loss: 3.6594e-04 - mean_squared_error: 3.6594e-04     
Epoch 156/500
537/537 [==============================] - 0s - loss: 3.5119e-04 - mean_squared_error: 3.5119e-04     
Epoch 157/500
537/537 [==============================] - 0s - loss: 3.6661e-04 - mean_squared_error: 3.6661e-04     
Epoch 158/500
537/537 [==============================] - 0s - loss: 3.6853e-04 - mean_squared_error: 3.6853e-04     
Epoch 159/500
537/537 [==============================] - 0s - loss: 3.6176e-04 - mean_squared_error: 3.6176e-04     
Epoch 160/500
537/537 [==============================] - 0s - loss: 3.6546e-04 - mean_squared_error: 3.6546e-04     
Epoch 161/500
537/537 [==============================] - 0s - loss: 4.0516e-04 - mean_squared_error: 4.0516e-04     
Epoch 162/500
537/537 [==============================] - 0s - loss: 4.0362e-04 - mean_squared_error: 4.0362e-04     
Epoch 163/500
537/537 [==============================] - 0s - loss: 3.2800e-04

537/537 [==============================] - 0s - loss: 2.4510e-04 - mean_squared_error: 2.4510e-04     
Epoch 226/500
537/537 [==============================] - 0s - loss: 1.8884e-04 - mean_squared_error: 1.8884e-04     
Epoch 227/500
537/537 [==============================] - 0s - loss: 1.8096e-04 - mean_squared_error: 1.8096e-04     
Epoch 228/500
537/537 [==============================] - 0s - loss: 2.1688e-04 - mean_squared_error: 2.1688e-04     
Epoch 229/500
537/537 [==============================] - 0s - loss: 1.8791e-04 - mean_squared_error: 1.8791e-04     
Epoch 230/500
537/537 [==============================] - 0s - loss: 1.6067e-04 - mean_squared_error: 1.6067e-04     
Epoch 231/500
537/537 [==============================] - 0s - loss: 1.6944e-04 - mean_squared_error: 1.6944e-04     
Epoch 232/500
537/537 [==============================] - 0s - loss: 1.9666e-04 - mean_squared_error: 1.9666e-04     
Epoch 233/500
537/537 [==============================] - 0s - loss: 1.5609e-04

537/537 [==============================] - 0s - loss: 2.5485e-04 - mean_squared_error: 2.5485e-04     
Epoch 296/500
537/537 [==============================] - 0s - loss: 2.5275e-04 - mean_squared_error: 2.5275e-04     
Epoch 297/500
537/537 [==============================] - 0s - loss: 2.7876e-04 - mean_squared_error: 2.7876e-04     
Epoch 298/500
537/537 [==============================] - 0s - loss: 2.3025e-04 - mean_squared_error: 2.3025e-04     
Epoch 299/500
537/537 [==============================] - 0s - loss: 2.2791e-04 - mean_squared_error: 2.2791e-04     
Epoch 300/500
537/537 [==============================] - 0s - loss: 2.4934e-04 - mean_squared_error: 2.4934e-04     
Epoch 301/500
537/537 [==============================] - 0s - loss: 2.0850e-04 - mean_squared_error: 2.0850e-04     
Epoch 302/500
537/537 [==============================] - 0s - loss: 1.8400e-04 - mean_squared_error: 1.8400e-04     
Epoch 303/500
537/537 [==============================] - 0s - loss: 1.7926e-04

537/537 [==============================] - 0s - loss: 1.3969e-04 - mean_squared_error: 1.3969e-04     
Epoch 366/500
537/537 [==============================] - 0s - loss: 2.0833e-04 - mean_squared_error: 2.0833e-04     
Epoch 367/500
537/537 [==============================] - 0s - loss: 3.1788e-04 - mean_squared_error: 3.1788e-04     
Epoch 368/500
537/537 [==============================] - 0s - loss: 3.7078e-04 - mean_squared_error: 3.7078e-04     
Epoch 369/500
537/537 [==============================] - 0s - loss: 3.4191e-04 - mean_squared_error: 3.4191e-04     
Epoch 370/500
537/537 [==============================] - 0s - loss: 3.0961e-04 - mean_squared_error: 3.0961e-04     
Epoch 371/500
537/537 [==============================] - 0s - loss: 3.4322e-04 - mean_squared_error: 3.4322e-04     
Epoch 372/500
537/537 [==============================] - 0s - loss: 4.4107e-04 - mean_squared_error: 4.4107e-04     
Epoch 373/500
537/537 [==============================] - 0s - loss: 3.5785e-04

537/537 [==============================] - 0s - loss: 1.5785e-04 - mean_squared_error: 1.5785e-04     
Epoch 436/500
537/537 [==============================] - 0s - loss: 3.1695e-04 - mean_squared_error: 3.1695e-04     
Epoch 437/500
537/537 [==============================] - 0s - loss: 3.2080e-04 - mean_squared_error: 3.2080e-04     
Epoch 438/500
537/537 [==============================] - 0s - loss: 2.7355e-04 - mean_squared_error: 2.7355e-04     
Epoch 439/500
537/537 [==============================] - 0s - loss: 4.1616e-04 - mean_squared_error: 4.1616e-04     
Epoch 440/500
537/537 [==============================] - 0s - loss: 5.7419e-04 - mean_squared_error: 5.7419e-04     
Epoch 441/500
537/537 [==============================] - 0s - loss: 6.6278e-04 - mean_squared_error: 6.6278e-04     
Epoch 442/500
537/537 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012     
Epoch 443/500
537/537 [==============================] - 0s - loss: 0.0017 - mean_squa

### Perceptron fuzzifier

### Training the fuzzifier

In [17]:
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print('\n Ensemble {} - \n'.format(k+1))
    X_feed_fw = X_train.copy()
    X_feed_re = X_train.copy()
    
    j = 0
    
    y_pred_fw = []
    y_pred_re = []
    
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        
        print('Getting op from chain node ',j)
        node_name = 'Node' + str(i)
        print('Shapes - \n X = {}'.format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)


 Ensemble 1 - 

Getting op from chain node  0
Shapes - 
 X = (537, 8)
Getting op from chain node  1
Shapes - 
 X = (537, 9)


In [18]:
forward_merged = np.array(y_full_fw).reshape(np.array(y_full_fw).shape[0]*np.array(y_full_fw).shape[1],np.array(y_full_fw).shape[2])
reverse_merged = np.array(y_full_re).reshape(np.array(y_full_re).shape[0]*np.array(y_full_re).shape[1],np.array(y_full_re).shape[2])

In [19]:
forward_merged[:,0]

array([0.4089533, 0.3016076], dtype=float32)

In [20]:
full_set = np.append(forward_merged,reverse_merged,axis=0).T
full_set.shape

(537, 4)

### Creating FCNN fuzzifier

In [21]:
fuzzifier = Sequential()
fuzzifier.add(Dense(y_train.shape[1],activation='linear',input_dim = full_set.shape[1]))
fuzzifier.compile(optimizer='adam',loss='mse')
fuzzifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [22]:
#Training Fuzzifier
fuzzifier.fit(full_set,y_train,epochs=500,batch_size=5)

Epoch 1/500
537/537 [==============================] - 0s - loss: 2.8210     
Epoch 2/500
537/537 [==============================] - 0s - loss: 1.7126     
Epoch 3/500
537/537 [==============================] - 0s - loss: 0.9824     
Epoch 4/500
537/537 [==============================] - 0s - loss: 0.5260     
Epoch 5/500
537/537 [==============================] - 0s - loss: 0.2613     
Epoch 6/500
537/537 [==============================] - 0s - loss: 0.1192     
Epoch 7/500
537/537 [==============================] - 0s - loss: 0.0505     
Epoch 8/500
537/537 [==============================] - 0s - loss: 0.0199     
Epoch 9/500
537/537 [==============================] - 0s - loss: 0.0076     
Epoch 10/500
537/537 [==============================] - 0s - loss: 0.0031     
Epoch 11/500
537/537 [==============================] - 0s - loss: 0.0017     
Epoch 12/500
537/537 [==============================] - 0s - loss: 0.0012     
Epoch 13/500
537/537 [==============================] - 0s - 

537/537 [==============================] - 0s - loss: 2.4100e-04     
Epoch 98/500
537/537 [==============================] - 0s - loss: 2.4302e-04     
Epoch 99/500
537/537 [==============================] - 0s - loss: 2.4620e-04     
Epoch 100/500
537/537 [==============================] - 0s - loss: 2.3648e-04     
Epoch 101/500
537/537 [==============================] - 0s - loss: 2.2842e-04     
Epoch 102/500
537/537 [==============================] - 0s - loss: 2.3648e-04     
Epoch 103/500
537/537 [==============================] - 0s - loss: 2.3868e-04     
Epoch 104/500
537/537 [==============================] - 0s - loss: 2.3975e-04     
Epoch 105/500
537/537 [==============================] - 0s - loss: 2.3997e-04     
Epoch 106/500
537/537 [==============================] - 0s - loss: 2.2993e-04     
Epoch 107/500
537/537 [==============================] - 0s - loss: 2.3213e-04     
Epoch 108/500
537/537 [==============================] - 0s - loss: 2.2796e-04     
Epoch 10

537/537 [==============================] - 0s - loss: 1.9407e-04     
Epoch 194/500
537/537 [==============================] - 0s - loss: 1.8951e-04     
Epoch 195/500
537/537 [==============================] - 0s - loss: 1.8924e-04     
Epoch 196/500
537/537 [==============================] - 0s - loss: 1.9413e-04     
Epoch 197/500
537/537 [==============================] - 0s - loss: 1.9333e-04     
Epoch 198/500
537/537 [==============================] - 0s - loss: 1.8979e-04     
Epoch 199/500
537/537 [==============================] - 0s - loss: 1.8715e-04     
Epoch 200/500
537/537 [==============================] - 0s - loss: 1.9170e-04     
Epoch 201/500
537/537 [==============================] - 0s - loss: 1.9048e-04     
Epoch 202/500
537/537 [==============================] - 0s - loss: 1.8868e-04     
Epoch 203/500
537/537 [==============================] - 0s - loss: 1.8786e-04     
Epoch 204/500
537/537 [==============================] - ETA: 0s - loss: 1.9067e-0 - 0s - 

537/537 [==============================] - 0s - loss: 1.7893e-04     
Epoch 291/500
537/537 [==============================] - 0s - loss: 1.7693e-04     
Epoch 292/500
537/537 [==============================] - 0s - loss: 1.7740e-04     
Epoch 293/500
537/537 [==============================] - 0s - loss: 1.7643e-04     
Epoch 294/500
537/537 [==============================] - 0s - loss: 1.7623e-04     
Epoch 295/500
537/537 [==============================] - 0s - loss: 1.7636e-04     
Epoch 296/500
537/537 [==============================] - 0s - loss: 1.7128e-04     
Epoch 297/500
537/537 [==============================] - 0s - loss: 1.7922e-04     
Epoch 298/500
537/537 [==============================] - 0s - loss: 1.7586e-04     
Epoch 299/500
537/537 [==============================] - 0s - loss: 1.7687e-04     
Epoch 300/500
537/537 [==============================] - 0s - loss: 1.8038e-04     
Epoch 301/500
537/537 [==============================] - 0s - loss: 1.7500e-04     
Epoch 

537/537 [==============================] - 0s - loss: 1.7337e-04     
Epoch 388/500
537/537 [==============================] - 0s - loss: 1.6335e-04     
Epoch 389/500
537/537 [==============================] - 0s - loss: 1.6654e-04     
Epoch 390/500
537/537 [==============================] - 0s - loss: 1.6714e-04     
Epoch 391/500
537/537 [==============================] - 0s - loss: 1.6620e-04     
Epoch 392/500
537/537 [==============================] - 0s - loss: 1.6487e-04     
Epoch 393/500
537/537 [==============================] - 0s - loss: 1.6528e-04     
Epoch 394/500
537/537 [==============================] - 0s - loss: 1.7757e-04     
Epoch 395/500
537/537 [==============================] - 0s - loss: 1.6894e-04     
Epoch 396/500
537/537 [==============================] - 0s - loss: 1.7234e-04     
Epoch 397/500
537/537 [==============================] - 0s - loss: 1.6490e-04     
Epoch 398/500
537/537 [==============================] - 0s - loss: 1.7143e-04     
Epoch 

537/537 [==============================] - 0s - loss: 1.6492e-04     
Epoch 485/500
537/537 [==============================] - 0s - loss: 1.6394e-04     
Epoch 486/500
537/537 [==============================] - 0s - loss: 1.5899e-04     
Epoch 487/500
537/537 [==============================] - 0s - loss: 1.6331e-04     
Epoch 488/500
537/537 [==============================] - 0s - loss: 1.6046e-04     
Epoch 489/500
537/537 [==============================] - 0s - loss: 1.6470e-04     
Epoch 490/500
537/537 [==============================] - 0s - loss: 1.6444e-04     
Epoch 491/500
537/537 [==============================] - 0s - loss: 1.5622e-04     
Epoch 492/500
537/537 [==============================] - 0s - loss: 1.6796e-04     
Epoch 493/500
537/537 [==============================] - 0s - loss: 1.6394e-04     
Epoch 494/500
537/537 [==============================] - 0s - loss: 1.6440e-04     
Epoch 495/500
537/537 [==============================] - 0s - loss: 1.6117e-04     
Epoch 

### Testing FCNN fuzzifier

In [23]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
        
## Merged Ensemble Reverse
forward_merged = np.array(y_full_fw).reshape(np.array(y_full_fw).shape[0]*np.array(y_full_fw).shape[1],
                           np.array(y_full_fw).shape[2])

## Merged Ensemble Reverse
reverse_merged = np.array(y_full_re).reshape(np.array(y_full_re).shape[0]*np.array(y_full_re).shape[1],
                           np.array(y_full_re).shape[2])

full_set_test = np.append(forward_merged,reverse_merged,axis=0).T
print("\n\nTest shape:, ",full_set_test.shape)



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (231, 8)
Getting op from chain node  1
Shapes:
 X = (231, 9)


Test shape:,  (231, 4)


In [24]:
predictions = fuzzifier.predict(full_set_test)
print(predictions.shape)

(231, 2)


In [25]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

RMSE:  1.1944833537183859
MAE:  0.8543172981625513
R_Squared:  0.9836223920770424
Adjusted R-squared -  0.9834497690686318


### Convolutional Fuzzifier

In [26]:
##Getting the output from the ensembles on training data
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_train.copy()
    X_feed_re = X_train.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
y_full_fw = np.array(y_full_fw)[:,:,:,0]
y_full_re = np.array(y_full_re)[:,:,:,0]




ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (537, 8)
Getting op from chain node  1
Shapes:
 X = (537, 9)


In [27]:
intervened = []
for forward,reverse in zip(y_full_fw,y_full_re):
    forward = forward.T
    reverse = reverse.T
    data = []
    for i in range(forward.shape[0]):
        row = []
        flipped_reversed = list(reverse[i])[::-1]
        for j in range(forward.shape[1]):
            row.append(forward[i,j])
            row.append(flipped_reversed[j])
        
        data.append(row)
    intervened.append(data)
intervened = np.array(intervened)

In [28]:
full_set = intervened[0,:,:]
for i in range(1,np.array(intervened).shape[0]):
    full_set = np.append(full_set,intervened[i,:,:],axis = 1)

In [29]:
from keras.layers import Conv1D,Flatten
#CNN Fuzzifier 
fuzzifier = Sequential()
fuzzifier.add(Conv1D(filters=1,kernel_size=2,strides=2,activation='relu',input_shape=(full_set.shape[1],1)))
fuzzifier.add(Flatten())
fuzzifier.add(Dense(128,activation='relu'))
fuzzifier.add(Dense(y_train.shape[1],activation='linear'))
fuzzifier.compile(optimizer='adam',loss='mse')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [30]:
fuzzifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2, 1)              3         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 128)               384       
_________________________________________________________________
dense_33 (Dense)             (None, 2)                 258       
Total params: 645
Trainable params: 645
Non-trainable params: 0
_________________________________________________________________


In [31]:
#Training Fuzzifier
fuzzifier.fit(full_set.reshape(full_set.shape[0],full_set.shape[1],1),y_train,epochs=500,batch_size=5)

Epoch 1/500
537/537 [==============================] - 1s - loss: 0.7247     
Epoch 2/500
537/537 [==============================] - 0s - loss: 0.2901     
Epoch 3/500
537/537 [==============================] - 0s - loss: 0.1336     
Epoch 4/500
537/537 [==============================] - 0s - loss: 0.0765     
Epoch 5/500
537/537 [==============================] - 0s - loss: 0.0395     
Epoch 6/500
537/537 [==============================] - 0s - loss: 0.0254     
Epoch 7/500
537/537 [==============================] - 0s - loss: 0.0201     
Epoch 8/500
537/537 [==============================] - 0s - loss: 0.0171     
Epoch 9/500
537/537 [==============================] - 0s - loss: 0.0148     
Epoch 10/500
537/537 [==============================] - 0s - loss: 0.0134     
Epoch 11/500
537/537 [==============================] - 0s - loss: 0.0121     
Epoch 12/500
537/537 [==============================] - 0s - loss: 0.0107     
Epoch 13/500
537/537 [==============================] - 0s - 

537/537 [==============================] - 0s - loss: 8.3931e-04     
Epoch 105/500
537/537 [==============================] - 0s - loss: 8.5582e-04     
Epoch 106/500
537/537 [==============================] - 0s - loss: 9.3045e-04     
Epoch 107/500
537/537 [==============================] - 0s - loss: 8.7312e-04     
Epoch 108/500
537/537 [==============================] - 0s - loss: 0.0011     
Epoch 109/500
537/537 [==============================] - 0s - loss: 8.5622e-04     
Epoch 110/500
537/537 [==============================] - 0s - loss: 8.1656e-04     
Epoch 111/500
537/537 [==============================] - 0s - loss: 8.2510e-04     
Epoch 112/500
537/537 [==============================] - 0s - loss: 7.9515e-04     
Epoch 113/500
537/537 [==============================] - 0s - loss: 8.5724e-04     
Epoch 114/500
537/537 [==============================] - 0s - loss: 7.6319e-04     
Epoch 115/500
537/537 [==============================] - 0s - loss: 7.8019e-04     
Epoch 116/

537/537 [==============================] - 0s - loss: 2.9274e-04     
Epoch 201/500
537/537 [==============================] - 0s - loss: 2.3884e-04     
Epoch 202/500
537/537 [==============================] - 0s - loss: 2.3426e-04     
Epoch 203/500
537/537 [==============================] - 0s - loss: 2.5049e-04     
Epoch 204/500
537/537 [==============================] - 0s - loss: 2.4699e-04     
Epoch 205/500
537/537 [==============================] - ETA: 0s - loss: 3.0880e-0 - 0s - loss: 3.0668e-04     
Epoch 206/500
537/537 [==============================] - 0s - loss: 2.5049e-04     
Epoch 207/500
537/537 [==============================] - 0s - loss: 2.4571e-04     
Epoch 208/500
537/537 [==============================] - 0s - loss: 2.1945e-04     
Epoch 209/500
537/537 [==============================] - 0s - loss: 2.3710e-04     
Epoch 210/500
537/537 [==============================] - 0s - loss: 3.0924e-04     
Epoch 211/500
537/537 [==============================] - 0s - 

537/537 [==============================] - 0s - loss: 2.6043e-04     
Epoch 298/500
537/537 [==============================] - 0s - loss: 2.2502e-04     
Epoch 299/500
537/537 [==============================] - 0s - loss: 2.3868e-04     
Epoch 300/500
537/537 [==============================] - 0s - loss: 2.3213e-04     
Epoch 301/500
537/537 [==============================] - 0s - loss: 2.1484e-04     
Epoch 302/500
537/537 [==============================] - 0s - loss: 2.5076e-04     
Epoch 303/500
537/537 [==============================] - 0s - loss: 3.0612e-04     
Epoch 304/500
537/537 [==============================] - 0s - loss: 2.0115e-04     
Epoch 305/500
537/537 [==============================] - 0s - loss: 2.2059e-04     
Epoch 306/500
537/537 [==============================] - 0s - loss: 2.4689e-04     
Epoch 307/500
537/537 [==============================] - 0s - loss: 2.4065e-04     
Epoch 308/500
537/537 [==============================] - 0s - loss: 2.5079e-04     
Epoch 

537/537 [==============================] - 0s - loss: 2.4966e-04     
Epoch 395/500
537/537 [==============================] - 0s - loss: 2.7680e-04     
Epoch 396/500
537/537 [==============================] - 0s - loss: 2.3961e-04     
Epoch 397/500
537/537 [==============================] - 0s - loss: 2.3542e-04     
Epoch 398/500
537/537 [==============================] - 0s - loss: 2.6247e-04     
Epoch 399/500
537/537 [==============================] - 0s - loss: 2.5596e-04     
Epoch 400/500
537/537 [==============================] - 0s - loss: 2.5579e-04     
Epoch 401/500
537/537 [==============================] - 0s - loss: 2.6362e-04     
Epoch 402/500
537/537 [==============================] - 0s - loss: 2.3401e-04     
Epoch 403/500
537/537 [==============================] - 0s - loss: 2.2830e-04     
Epoch 404/500
537/537 [==============================] - 0s - loss: 2.5941e-04     
Epoch 405/500
537/537 [==============================] - 0s - loss: 2.0610e-04     
Epoch 

537/537 [==============================] - 0s - loss: 2.5173e-04     
Epoch 491/500
537/537 [==============================] - 0s - loss: 2.3370e-04     
Epoch 492/500
537/537 [==============================] - 0s - loss: 2.5916e-04     
Epoch 493/500
537/537 [==============================] - 0s - loss: 2.5369e-04     
Epoch 494/500
537/537 [==============================] - 0s - loss: 2.5764e-04     
Epoch 495/500
537/537 [==============================] - 0s - loss: 2.0972e-04     
Epoch 496/500
537/537 [==============================] - 0s - loss: 2.2817e-04     
Epoch 497/500
537/537 [==============================] - 0s - loss: 2.8491e-04     
Epoch 498/500
537/537 [==============================] - 0s - loss: 2.3499e-04     
Epoch 499/500
537/537 [==============================] - 0s - loss: 2.4600e-04     
Epoch 500/500
537/537 [==============================] - 0s - loss: 2.5972e-04     


### Testing CNN Fuzzifier

In [32]:
##Getting the output from the ensembles on training data
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
y_full_fw = np.array(y_full_fw)[:,:,:,0]
y_full_re = np.array(y_full_re)[:,:,:,0]




ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (231, 8)
Getting op from chain node  1
Shapes:
 X = (231, 9)


In [33]:
intervened = []
for forward,reverse in zip(y_full_fw,y_full_re):
    forward = forward.T
    reverse = reverse.T
    data = []
    for i in range(forward.shape[0]):
        row = []
        flipped_reversed = list(reverse[i])[::-1]
        for j in range(forward.shape[1]):
            row.append(forward[i,j])
            row.append(flipped_reversed[j])
        
        data.append(row)
    intervened.append(data)
intervened = np.array(intervened)

In [34]:
full_set_test = intervened[0,:,:]
for i in range(1,np.array(intervened).shape[0]):
    full_set_test = np.append(full_set_test,intervened[i,:,:],axis = 1)

In [35]:
full_set_test = full_set_test.reshape(full_set_test.shape[0],full_set_test.shape[1],1)
predictions = fuzzifier.predict(full_set_test)
print(predictions.shape)

(231, 2)


In [36]:
evaluate(scaler.inverse_transform(y_test),
         scaler.inverse_transform(np.array(predictions)))

RMSE:  1.2830292672736647
MAE:  0.8921444439578365
R_Squared:  0.980888659536632
Adjusted R-squared -  0.9806872224829997


### Mean Aggregation

In [37]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (231, 8)
Getting op from chain node  1
Shapes:
 X = (231, 9)


In [38]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [39]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

### Evaluating Mean Aggregation

In [40]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

RMSE:  1.2548418662303766
MAE:  0.8634271034637053
R_Squared:  0.9819496564729489
Adjusted R-squared -  0.9817594025227296
